In [1]:
import pandas as pd
import yfinance as yf
import datetime

In [2]:
df_opt = pd.read_csv('Optimizer2.csv')
df_opt

,Unnamed: 0,SPY,QQQ,DIA,TLT,IEF
0,bucket = 1,-0.0000,0.3567,0.3292,0.1351,-0.1790
1,bucket = 2,-0.0006,0.2772,-0.0002,0.0003,0.7216
2,bucket = 3,0.1509,0.1788,-0.3524,0.3170,0.0008


In [3]:
df_pred = pd.read_csv('Predictions_test.csv')
df_pred

,Unnamed: 0,Date,VIX,bucket_pred,rawbucket,maxbucket
0,0,2003-01-02,28.740000,1,2,2
1,1,2003-01-03,25.549999,1,2,2
2,2,2003-01-06,25.320000,3,2,3
3,3,2003-01-07,25.129999,3,2,3
4,4,2003-01-08,25.620001,3,2,3
...,...,...,...,...,...,...
4419,392,2020-07-23,7190.999700,1,2,2
4420,393,2020-07-24,8387.999700,1,2,2
4421,394,2020-07-27,7980.000000,1,2,2
4422,395,2020-07-28,7458.000300,1,2,2


In [4]:
# fetch close and open price data
tickers = ['SPY','QQQ','DIA','TLT','IEF']
start_date = min(df_pred['Date'])
end_date = str(pd.to_datetime(max(df_pred['Date']))+datetime.timedelta(days=2))[:10]
assets = yf.download(tickers,start_date,end_date)

[*********************100%***********************]  5 of 5 completed


In [6]:
# df: dataframe to store final results
df = pd.DataFrame(columns=['Date','VIX','maxbucket']+[i+'_weight' for i in tickers])
df[['Date','VIX','maxbucket']] = df_pred[['Date','VIX','maxbucket']]

# add weight data to df
for i in range(df.shape[0]):
    df.loc[i,[i+'_weight' for i in tickers]] = list(df_opt.iloc[df['maxbucket'][i]-1,1:])

# add close price and open price data to df   
df[[i+'_close' for i in tickers]] = assets['Close'][tickers].iloc[0:-1,:].reset_index(drop=True)
df[[i+'_open' for i in tickers]] = assets['Open'][tickers].shift(-1).iloc[0:-1,:].reset_index(drop=True)

# calculate return
df['return'] = sum([(df[i+'_open']-df[i+'_close'])/df[i+'_close']*df[i+'_weight'] for i in tickers])
df

,Date,VIX,maxbucket,SPY_weight,QQQ_weight,DIA_weight,TLT_weight,IEF_weight,SPY_close,QQQ_close,DIA_close,TLT_close,IEF_close,SPY_open,QQQ_open,DIA_open,TLT_open,IEF_open,return
0,2003-01-02,28.740000,2,-0.0006,0.2772,-0.0002,0.0003,0.7216,91.070000,25.400000,86.180000,86.279999,84.790001,90.910004,25.459999,86.000000,86.150002,84.699997,-0.000110156
1,2003-01-03,25.549999,2,-0.0006,0.2772,-0.0002,0.0003,0.7216,91.349998,25.680000,86.360001,86.480003,84.930000,91.239998,25.709999,86.300003,86.169998,84.800003,-0.000780905
2,2003-01-06,25.320000,3,0.1509,0.1788,-0.3524,0.317,0.0008,92.959999,26.320000,87.669998,86.250000,84.720001,92.900002,26.410000,87.650002,86.190002,84.720001,0.000373873
3,2003-01-07,25.129999,3,0.1509,0.1788,-0.3524,0.317,0.0008,92.730003,26.650000,87.720001,86.559998,84.989998,92.199997,26.389999,87.199997,86.879997,85.169998,0.000655751
4,2003-01-08,25.620001,3,0.1509,0.1788,-0.3524,0.317,0.0008,91.389999,25.900000,86.169998,86.989998,85.190002,91.820000,26.240000,86.650002,86.169998,84.580002,-0.00189972
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4419,2020-07-23,7190.999700,2,-0.0006,0.2772,-0.0002,0.0003,0.7216,322.959991,258.010010,266.690002,169.750000,122.400002,320.950012,254.039993,265.149994,169.009995,122.300003,-0.00485125
4420,2020-07-24,8387.999700,2,-0.0006,0.2772,-0.0002,0.0003,0.7216,320.880005,255.559998,264.690002,169.740005,122.339996,321.630005,257.260010,264.769989,170.250000,122.440002,0.00243327
4421,2020-07-27,7980.000000,2,-0.0006,0.2772,-0.0002,0.0003,0.7216,323.220001,260.119995,265.950012,169.110001,122.150002,322.429993,259.309998,264.920013,169.539993,122.300003,2.59529e-05
4422,2020-07-28,7458.000300,2,-0.0006,0.2772,-0.0002,0.0003,0.7216,321.170013,256.809998,263.929993,170.279999,122.440002,322.119995,258.399994,264.190002,170.050003,122.449997,0.00177276


In [8]:
# avg return overnight
print('Avg return overnight \t',sum(df['return'])/df.shape[0])
print('Avg loss overnight \t',sum(df['return'][df['return']<0])/df.shape[0])
print('Largest loss overnight \t',min(df['return'][df['return']<0]))

Avg return overnight 	 0.0001830947247433947
Avg loss overnight 	 -0.0007322176987132427
Largest loss overnight 	 -0.016372480583562218
